## Package Imports

In [ ]:
import pyaudio
import wave

## Variable Initializations

In [ ]:
audio_format = pyaudio.paInt16
audio_rate = 44100
audio_channels = 1
audio_device_index = 1
audio_frames_per_buffer = 4096
audio_sample_duration = 2

## Processing Microphone Audio

In [ ]:
pa = pyaudio.PyAudio()
    
stream = pa.open(format = audio_format,
                 rate = audio_rate,
                 channels = audio_channels,
                 input_device_index = audio_device_index,
                 frames_per_buffer = audio_frames_per_buffer,
                 input = True)

print("Now recording audio...")
frames = []

# loop through stream and append audio chunks to frame array
for i in range(0, int((audio_rate / audio_frames_per_buffer) * audio_sample_duration)):
    data = stream.read(audio_frames_per_buffer)
    frames.append(data)

print("Finished recording audio...")

# stop the stream, close it, and terminate the pyaudio instantiation
stream.stop_stream()
stream.close()
pa.terminate()

# save the audio frames as .wav file
wavefile = wave.open("test",'wb')
wavefile.setnchannels(audio_channels)
wavefile.setsampwidth(pa.get_sample_size(audio_format))
wavefile.setframerate(audio_rate)
wavefile.writeframes(b''.join(frames))
wavefile.close()

# while True:
#     try:
#         block = stream.read(audio_input_frames_per_block)
#         print(block)
#     except:
#         print("--- Error Trying to Process Microphone Audio ---")
#         pass